# COVID - Ocupação Hospitalar (2020) 
## - - - - *em progresso* - - - -

Utilizando dados fornecidos pelo Ministério da Saúde, iremos analisar as **variações na ocupação hospitalar no país, com *foco em ocorrências de COVID-19*, para o ano 2020.**
Os conjuntos de dados para cada ano podem ser baixados no formato `.csv` através deste [link](https://dados.gov.br/dados/conjuntos-dados/registro-de-ocupacao-hospitalar-covid-19). O link redireciona para uma página específica dentro do **[Portal Brasileiro de Dados Abertos](https://dados.gov.br/home)**

No entanto, **não há documentação disponível**. As únicas informações disponibilizadas a respeito dos dados são:
>"O módulo Internações foi desenvolvido para registro da ocupação de leitos clínicos e de Unidade de Terapia Intensiva (UTI) SUS destinados para atendimento aos pacientes com casos suspeitos ou confirmados da COVID-19 (ocupação SRAG / COVID-19)"

e também:

>"A partir do ano de 2022 foram acrescentado novos campos utilizados para descrever a ocupação dos leitos. Registros com ano anterior a 2022 não contém esses campos preenchidos."

Com essa base, vamos explorar o conjunto de dados para entender melhor o que temos em mãos.

> Dados obtidos em 17/04/2023

## Data Wrangling

### Visão Geral

In [1]:
import pandas as pd
import numpy as np
import utils

In [2]:
df = pd.read_csv('ocupacao-leito-2020.csv')

/tmp/ipykernel_80752/2661094983.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('ocupacao-leito-2020.csv')


Já somos recebidos com um aviso indicando que há uma coluna com tipos de dados misturados, mas vamos além para investigar o restante e conhecer melhor nosso material de análise.

Para isso será útil verificar uma amostra do conjunto de dados e listar todas as colunas junto com seus respectivos tipos de dados.

In [3]:
df.head()

,Unnamed: 0,_id,dataNotificacao,cnes,ocupacaoSuspeitoCli,ocupacaoSuspeitoUti,ocupacaoConfirmadoCli,ocupacaoConfirmadoUti,ocupacaoCovidUti,ocupacaoCovidCli,...,origem,_p_usuario,estadoNotificacao,municipioNotificacao,estado,municipio,excluido,validado,_created_at,_updated_at
0,0,p5Ez41Zu6B,2020-04-14T03:00:00.000Z,2303167,0.0,0.0,0.0,0.0,NaN,NaN,...,aplicacao-web,_User$auVMjEVEVz,Santa Catarina,Itapema,Santa Catarina,Itapema,True,False,2020-04-15T13:44:30.085Z,2020-04-16T13:56:13.656Z
1,1,eUOMsSt7T7,2020-04-15T03:00:00.000Z,5935377\t,1.0,0.0,0.0,0.0,NaN,NaN,...,parse-cloud,_User$WBa5nfe9P9,Rio de Janeiro,Rio de Janeiro,Rio de Janeiro,Rio de Janeiro,False,False,2020-04-15T13:50:47.256Z,2020-08-01T20:08:13.116Z
2,2,QaG4oIBrMa,2020-04-15T14:57:16.255Z,0012599,15.0,1.0,3.0,2.0,NaN,NaN,...,parse-cloud,_User$6oxZQHBPQ7,Rio de Janeiro,Niterói,Rio de Janeiro,Niterói,False,False,2020-04-15T15:20:57.033Z,2020-08-01T20:02:12.729Z
3,3,ifya31F3ZF,2020-03-24T03:00:00.000Z,2562871,9.0,0.0,1.0,0.0,NaN,NaN,...,parse-cloud,_User$fhufIbmR9C,Ceará,Várzea Alegre,Ceará,Várzea Alegre,False,False,2020-04-15T16:58:32.839Z,2020-08-01T21:40:37.624Z
4,4,dkurDhelx9,2020-04-01T03:00:00.000Z,2252228,26.0,0.0,1.0,0.0,NaN,NaN,...,parse-cloud,_User$tkAdHTNhC9,Rio Grande do Sul,Encantado,Rio Grande do Sul,Encantado,False,False,2020-04-15T17:02:53.877Z,2020-08-01T21:37:53.664Z


In [4]:
df.dtypes

Unnamed: 0                 int64
_id                       object
dataNotificacao           object
cnes                      object
ocupacaoSuspeitoCli      float64
ocupacaoSuspeitoUti      float64
ocupacaoConfirmadoCli    float64
ocupacaoConfirmadoUti    float64
ocupacaoCovidUti         float64
ocupacaoCovidCli         float64
ocupacaoHospitalarUti    float64
ocupacaoHospitalarCli    float64
saidaSuspeitaObitos      float64
saidaSuspeitaAltas       float64
saidaConfirmadaObitos    float64
saidaConfirmadaAltas     float64
origem                    object
_p_usuario                object
estadoNotificacao         object
municipioNotificacao      object
estado                    object
municipio                 object
excluido                    bool
validado                    bool
_created_at               object
_updated_at               object
dtype: object

### Deduzindo Colunas

#### `cnes` e valores privados *(`_...`)*

A coluna sobre a qual recebemos o aviso se trata de `cnes`, referente ao *Cadastro Nacional de Estabelecimentos de Saúde*. Pelo que é possível verificar, deveriamos ver apenas números, mas aparentemente existem erros de digitação inserindo caractéres em alguns registros.

Além disso, é possível verificar alguns campos iniciados com "**_**", que geralmente indicam *variáveis privadas voltadas para uso interno de alguma aplicação*. **Possivelmente não acrescentarão muito para a análise**, mas é importante verificar se de fato podemos excluí-los sem perdas.

Analisando os primeiros registros é possível notar que `_created_at` parece ser a data em que o registro foi criado. Muitos deles são feitos em sequência num *curto período de tempo*, adicionando registros `dataNotificacao` de datas passadas. 

`_updated_at` apresenta padrão semelhante (inserção em lotes), provavelmente gerando uma análise desconexa. Logo, `dataNotificacao` seria a infomação efetivamente útil para nossa análise. 

`_p_usuario` proavelmente se refere ao 'id' do usuário que inseriu o registro, enquanto `_id` possivelmente se trata do identificador do registro em si. Podemos verificar se a quantidade de `_id` únicos é igual ao número total de registros.

In [5]:
shape = df.shape
shape[0] == len(df['_id'].unique())

True

`_id` é mesmo o que imaginamos. A propósito, podemos aproveitar e conferir com quantos registros estamos lidando:

In [6]:
print(f"Nosso DataFrame possui {shape[0]} linhas e {shape[1]} colunas")

Nosso DataFrame possui 554687 linhas e 26 colunas


Vamos começar a listar todas as colunas que não terão utilidade para a análise.

In [7]:
to_drop = ['_id', '_p_usuario', '_created_at', '_updated_at']

#### Campos Novos - *Utilizados apenas a partir de 2022*

Agora podemos prosseguir para as demais colunas. 

Logo de início podemos notar que nas 5 primeiras linhas, todos os registros de `ocupacaoCovidUti` e `ocupacaoCovidCli` não estão preenchidos. Será que isso se mantém para todo o documento?

In [8]:
df['ocupacaoCovidUti'].value_counts()

ocupacaoCovidUti
 0.0     336
 20.0     67
 19.0     41
 1.0      21
 18.0     16
 2.0       5
 3.0       4
 17.0      3
 10.0      3
 6.0       3
 7.0       2
 5.0       2
 4.0       1
-1.0       1
 8.0       1
 44.0      1
 22.0      1
 25.0      1
 23.0      1
 11.0      1
Name: count, dtype: int64

Dos mais de 550 mil registros no documento, na coluna `ocupacaoCovidUti` somente cerca de 500 possuem um valor válido (incluindo um valor negativo).

Por padrão, o método `.head()` não mostra todas as colunas, então é possível que além de `ocupacaoCovidUti` e `ocupacaoCovidCli` outras colunas apresentem o mesmo padrão. 

Mesmo que este arquivo seja referente ao ano 2020, vimos que existe um campo específico para data de *atualização do registro*. Além disso, segundo a fonte, *registros a partir de 2022 possuem novos campos*. Dessa maneira, **é possível que dados atualizados depois de 2022 apresentem inconsistências**.

Podemos começar a investigar essa hipótese verificando melhor as estatísticas descritivas.

In [9]:
df.describe()

,Unnamed: 0,ocupacaoSuspeitoCli,ocupacaoSuspeitoUti,ocupacaoConfirmadoCli,ocupacaoConfirmadoUti,ocupacaoCovidUti,ocupacaoCovidCli,ocupacaoHospitalarUti,ocupacaoHospitalarCli,saidaSuspeitaObitos,saidaSuspeitaAltas,saidaConfirmadaObitos,saidaConfirmadaAltas
count,5.546870e+05,5.542860e+05,554286.000000,554287.000000,554287.000000,511.000000,511.000000,511.000000,511.000000,554571.000000,5.545830e+05,554675.000000,5.546740e+05
mean,2.802098e+05,1.901152e+02,7.139670,6.081243,4.477320,5.301370,4.651663,7.043053,22.080235,0.224539,5.138644e+01,0.373017,3.395598e+01
std,1.677398e+05,6.208808e+04,16.098472,33.178988,11.254881,8.597718,5.848411,12.397110,51.228089,3.571056,3.766872e+04,3.744652,1.729309e+04
min,0.000000e+00,-5.700000e+01,-9.000000,-10.000000,-7.000000,-1.000000,0.000000,0.000000,0.000000,-10.000000,-1.200000e+01,-3.000000,-1.300000e+01
25%,1.387045e+05,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00
50%,2.773850e+05,4.000000e+00,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,8.000000,0.000000,0.000000e+00,0.000000,0.000000e+00
75%,4.160795e+05,2.700000e+01,8.000000,6.000000,5.000000,17.500000,10.000000,20.000000,11.000000,0.000000,0.000000e+00,0.000000,0.000000e+00
max,1.618772e+06,2.908202e+07,924.000000,22222.000000,334.000000,44.000000,60.000000,195.000000,314.000000,425.000000,2.805202e+07,322.000000,9.112020e+06


Constatamos exatamente 511 registros nas colunas `ocupacaoCovidUti`, `ocupacaoCovidCli`, `ocupacaoHospitalarUti`, `ocupacaoHospitalarCli`, o que corrobora para nossa hipótese de registros atualizados depois de 2022.

Para confirmar, podemos averiguar as datas de atualização dos registros de qualquer um dos 4 campos em questão.

In [10]:
df.loc[df['ocupacaoCovidUti'].notnull(), '_updated_at']

14327     2022-02-21T17:57:08.292Z
55820     2022-02-21T18:02:56.533Z
103291    2021-10-05T13:25:13.346Z
104964    2021-10-05T13:26:12.193Z
113207    2021-10-05T13:26:45.066Z
                    ...           
554682    2023-02-15T19:28:39.477Z
554683    2023-03-08T12:03:10.135Z
554684    2023-04-06T14:28:58.666Z
554685    2023-04-12T12:57:54.509Z
554686    2023-04-12T12:58:12.391Z
Name: _updated_at, Length: 511, dtype: object

Considerando os 511 registros contestados, a atualização mais antiga ocorreu em 21/02/**2022**, e a mais recente em 12/04/**2023**, comprovando nossa hipótese.

Como não temos o conhecimento exato da informação que cada campo nos traz, a julgar pelo nome das colunas, parecem se tratar de *informações objetivamente diferentes*. Ainda, suas estatísticas não parecem diferir muito do resto do conjunto. Com essas considerações, *junto com o fato de representarem cerca de 0,09% dos dados*, acredito que seja melhor não incluí-las na análise.

Em vez de adicionarmos à primeira lista de remoção `to_drop`, vamos armazená-las em uma outra lista. Faremos isso pois irão ser removidas em etapas diferentes para facilitar nossa próxima análise das estatísticas descritivas.

In [11]:
to_drop_numeric = ['ocupacaoCovidUti', 'ocupacaoCovidCli', 'ocupacaoHospitalarUti', 'ocupacaoHospitalarCli']

#### `Unnamed: 0`

Aproveitando a criação de uma lista especial para dados numéricos, vamos checar a coluna `Unnamed: 0`.

Sendo do tipo `int64`, aparentemente se trata de um **índice**, identificando cada coluna do DataFrame com um número único. Podemos confirmar fazendo a contagem de valores únicos na coluna e comparando com o total de linhas do DataFrame.

In [12]:
len(df['Unnamed: 0'].unique()) == df.shape[0]

True

As quantidades bateram, o que comprova nossa hipótese. Aqui temos duas opções, incluir `Unnamed:  0` na lista para remoção ou usá-la como índice.

Como carregamos o arquivo *sem indicar* uma coluna para servir de índice, Pandas criou uma *automaticamente* no momento do carregamento. Já que efetivamente estamos apenas com algo **duplicado**, podemos optar pela remoção.

In [13]:
to_drop_numeric.append('Unnamed: 0')

#### `origem`, `excluido` e `validado`

O campo `origem` aparentemente indica a forma com que os dados foram coletados ou processados. Para ter mais certeza, podemos verificar quais os valores encontrados.

In [14]:
df['origem'].unique()

array(['aplicacao-web', 'parse-cloud', 'RPA-SP', 'RPA-CE', 'RPA-RS',
       'RPA-MS', 'RPA-GO', 'RPA-MT', 'RPA-RN', 'RPA-PR', 'RPA-MG',
       'RPA-MG-BELO-HORIZONTE', 'RPA-PR-CURITIBA'], dtype=object)

"RPA" provavelmente se refere à "Robotic Process Automation", enquanto "parse-cloud" pode se referir a um serviço de nuvem que faz a análise sintática (parsing) dos dados. De toda maneira, não é algo que será levado em consideração na análise de ocupação hospitalar.

In [15]:
to_drop.append('origem')

Veremos agora o que `excluido` e `validado` podem nos mostrar.

In [16]:
df['excluido'].value_counts()

excluido
False    554182
True        505
Name: count, dtype: int64

In [17]:
df.loc[df['excluido'] == True]


,Unnamed: 0,_id,dataNotificacao,cnes,ocupacaoSuspeitoCli,ocupacaoSuspeitoUti,ocupacaoConfirmadoCli,ocupacaoConfirmadoUti,ocupacaoCovidUti,ocupacaoCovidCli,...,origem,_p_usuario,estadoNotificacao,municipioNotificacao,estado,municipio,excluido,validado,_created_at,_updated_at
0,0,p5Ez41Zu6B,2020-04-14T03:00:00.000Z,2303167,0.0,0.0,0.0,0.0,NaN,NaN,...,aplicacao-web,_User$auVMjEVEVz,Santa Catarina,Itapema,Santa Catarina,Itapema,True,False,2020-04-15T13:44:30.085Z,2020-04-16T13:56:13.656Z
31,31,tv1uXbMbdc,2020-04-15T17:23:44.000Z,2240335,20.0,5.0,0.0,0.0,NaN,NaN,...,aplicacao-web,_User$Enseqp32PQ,Rio Grande do Sul,Farroupilha,Rio Grande do Sul,Farroupilha,True,False,2020-04-15T17:24:00.316Z,2020-04-15T17:35:07.999Z
62,62,fHqEA6b8Xt,2020-04-15T19:18:27.505Z,2375850,0.0,0.0,0.0,10.0,NaN,NaN,...,aplicacao-web,_User$Utr4cDP5eD,Mato Grosso do Sul,Paranaíba,Mato Grosso do Sul,Paranaíba,True,False,2020-04-15T19:29:13.561Z,2020-04-15T19:48:07.806Z
70,70,Zekk3kC7Gp,2020-04-01T03:00:00.000Z,2760819,1.0,0.0,0.0,0.0,NaN,NaN,...,aplicacao-web,_User$cIStY25tjj,Minas Gerais,Minduri,Minas Gerais,Minduri,True,False,2020-04-15T19:38:08.434Z,2020-04-15T19:49:49.157Z
72,72,6m0iH5blyQ,2020-04-02T03:00:00.000Z,2760819,1.0,0.0,0.0,0.0,NaN,NaN,...,aplicacao-web,_User$cIStY25tjj,Minas Gerais,Minduri,Minas Gerais,Minduri,True,False,2020-04-15T19:39:52.298Z,2020-04-15T19:50:00.013Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19617,19621,JO81FFRtz8,2020-05-11T13:11:48.671Z,2436450,133.0,28.0,9.0,8.0,NaN,NaN,...,aplicacao-web,_User$1XfNU4qbhz,Santa Catarina,Joinville,Santa Catarina,Joinville,True,False,2020-05-11T13:20:38.299Z,2020-05-12T11:09:21.900Z
19618,19622,EACElG3PDS,2020-05-11T13:11:48.671Z,2436450,133.0,28.0,9.0,8.0,NaN,NaN,...,aplicacao-web,_User$1XfNU4qbhz,Santa Catarina,Joinville,Santa Catarina,Joinville,True,False,2020-05-11T13:21:46.909Z,2020-05-12T11:09:30.965Z
19619,19623,S63bzGI2iK,2020-05-11T13:11:48.671Z,2436450,133.0,28.0,9.0,8.0,NaN,NaN,...,aplicacao-web,_User$1XfNU4qbhz,Santa Catarina,Joinville,Santa Catarina,Joinville,True,False,2020-05-11T13:20:47.607Z,2020-05-12T11:10:07.500Z
19813,19821,OZzcvODn1G,2020-05-11T15:38:05.414Z,2436450,133.0,28.0,9.0,8.0,NaN,NaN,...,aplicacao-web,_User$1XfNU4qbhz,Santa Catarina,Joinville,Santa Catarina,Joinville,True,False,2020-05-11T15:40:22.575Z,2020-05-12T11:09:17.290Z


Apesar de não haver documentação disponível, o nome da coluna em si (`excluido`) já indica fortemente que os registros marcados como verdadeiros não são aptos para uso nessa análise. 

Encontramos **505 registros** marcados, e destes podemos notar algumas particularidades como: 

Registros `dataNotificacao` exatamente iguais *(com precisão de 0,001 segundos)* para a mesma `cnes` e feitos pelo mesmo `_p_usuario`, vários deles com dados idênticos em `ocupacaoSuspeitoCli`, `ocupacaoSuspeitoUti`, `ocupacaoConfirmadoCli`, `ocupacaoConfirmadoUti`.

In [18]:
df.loc[(df['excluido'] == True) & (df.duplicated(subset=['dataNotificacao']))]

,Unnamed: 0,_id,dataNotificacao,cnes,ocupacaoSuspeitoCli,ocupacaoSuspeitoUti,ocupacaoConfirmadoCli,ocupacaoConfirmadoUti,ocupacaoCovidUti,ocupacaoCovidCli,...,origem,_p_usuario,estadoNotificacao,municipioNotificacao,estado,municipio,excluido,validado,_created_at,_updated_at
70,70,Zekk3kC7Gp,2020-04-01T03:00:00.000Z,2760819,1.0,0.0,0.0,0.0,NaN,NaN,...,aplicacao-web,_User$cIStY25tjj,Minas Gerais,Minduri,Minas Gerais,Minduri,True,False,2020-04-15T19:38:08.434Z,2020-04-15T19:49:49.157Z
72,72,6m0iH5blyQ,2020-04-02T03:00:00.000Z,2760819,1.0,0.0,0.0,0.0,NaN,NaN,...,aplicacao-web,_User$cIStY25tjj,Minas Gerais,Minduri,Minas Gerais,Minduri,True,False,2020-04-15T19:39:52.298Z,2020-04-15T19:50:00.013Z
73,73,J6qrjnb2y4,2020-04-03T03:00:00.000Z,2760819,1.0,0.0,0.0,0.0,NaN,NaN,...,aplicacao-web,_User$cIStY25tjj,Minas Gerais,Minduri,Minas Gerais,Minduri,True,False,2020-04-15T19:41:13.629Z,2020-04-15T19:50:20.836Z
74,74,84u09qrCH4,2020-04-04T03:00:00.000Z,2760819,0.0,0.0,0.0,0.0,NaN,NaN,...,aplicacao-web,_User$cIStY25tjj,Minas Gerais,Minduri,Minas Gerais,Minduri,True,False,2020-04-15T19:41:51.378Z,2020-04-15T19:50:07.477Z
186,186,lMXoydTmd1,2020-04-14T03:00:00.000Z,2418967,25.0,0.0,2.0,0.0,NaN,NaN,...,aplicacao-web,_User$1aBS46RcA0,Santa Catarina,São João Batista,Santa Catarina,São João Batista,True,False,2020-04-15T20:24:50.502Z,2020-04-15T21:42:10.108Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13772,13776,deIHZTOWsU,2020-04-02T03:00:00.000Z,0105724,0.0,0.0,0.0,0.0,NaN,NaN,...,aplicacao-web,_User$QtAd4NBwBc,Paraná,Corbélia,Paraná,Corbélia,True,False,2020-05-04T20:25:54.924Z,2020-05-04T20:26:32.391Z
18500,18504,8tidqNgXtv,2020-05-09T17:30:23.940Z,2436450,117.0,20.0,9.0,8.0,NaN,NaN,...,aplicacao-web,_User$1XfNU4qbhz,Santa Catarina,Joinville,Santa Catarina,Joinville,True,False,2020-05-09T18:34:15.464Z,2020-05-12T11:09:58.154Z
19618,19622,EACElG3PDS,2020-05-11T13:11:48.671Z,2436450,133.0,28.0,9.0,8.0,NaN,NaN,...,aplicacao-web,_User$1XfNU4qbhz,Santa Catarina,Joinville,Santa Catarina,Joinville,True,False,2020-05-11T13:21:46.909Z,2020-05-12T11:09:30.965Z
19619,19623,S63bzGI2iK,2020-05-11T13:11:48.671Z,2436450,133.0,28.0,9.0,8.0,NaN,NaN,...,aplicacao-web,_User$1XfNU4qbhz,Santa Catarina,Joinville,Santa Catarina,Joinville,True,False,2020-05-11T13:20:47.607Z,2020-05-12T11:10:07.500Z


Muitos registros de fato seguem o padrão que identificamos. Provavelmente são registros desatualizados ou com algum tipo de inconsistência.

Além do nome sugestivo da coluna, obtivemos várias informações que nos permitem ser **assertivos na eliminação de todos os registros em que `excluido` = `True`**. Como após a exclusão a coluna não terá mais serventia, podemos também eliminá-la dos dados para análise.

In [19]:
df.drop(df.loc[df['excluido'] == True].index, inplace=True)

In [20]:
to_drop.append('excluido')

Agora vejamos `validado`:

In [21]:
df['validado'].value_counts()

validado
False    554182
Name: count, dtype: int64

Todos os registros estão marcados como `False`, o que nos impossibilita buscar por padrões para inferir o que de fato a coluna significa.

Em busca de mais informações, encontrei outro [local onde o mesmo conjunto de dados é disponibilizado](https://opendatasus.saude.gov.br/dataset/registro-de-ocupacao-hospitalar-covid-19), o *OpenDataSUS*. Verifiquei que tanto o [Portal Brasileiro de Dados Abertos](https://dados.gov.br/home) *(onde obtivemos os dados)* quanto o [OpenDataSUS](https://opendatasus.saude.gov.br/) **apontam para o mesmo [URL](https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/LEITOS/2023-04-21/esus-vepi.LeitoOcupacao_2020.csv)** 
> *(o arquivo está armazenado em um bucket S3 da Amazon. Clicar no link 'URL' inicia o download)*. 

Ou seja, tanto um quanto outro nos fornecem o **mesmo arquivo**. Logo, qualquer informação encontrada em ambos endereços é igualmente válida para os dados obtidos.

No OpenDataSUS temos algumas [informações adicionais](https://opendatasus.saude.gov.br/dataset/registro-de-ocupacao-hospitalar-covid-19/resource/a16c8d5f-dc6d-46c8-8e75-41765a18a289) sobre o conjunto de dados em si. Temos informações como data de criação, **status de atividade** e **data da última modificação** *(a última presente em ambos)*. 

* Estado: ativo
* Dados atualizados pela última vez: 22/04/2023 *(dia em que foi feita a verificação)*

Também podemos encontrar registros do [fluxo de atividade](https://opendatasus.saude.gov.br/dataset/activity/registro-de-ocupacao-hospitalar-covid-19). Aqui vemos que os dados são **atualizados diariamente**.

Sabendo que se trata de um arquivo atualizado e mantido sob cuidado diário, podemos concluir que os dados em que estamos trabalhando são utilizáveis, ainda que o campo `validado` possa nos induzir a supor o contrário.

Dessa maneira, como todos os registros do arquivo possuem o mesmo valor, nada é agregado à análise. Iremos apenas *descartar a coluna*.

In [22]:
to_drop.append('validado')

#### `municipio` / `municipioNotificacao`, `estado` / `estadoNotificacao`

Ainda nos restam as colunas `municipio`, `estado`, `municipioNotificacao`, `estadoNotificacao`. Vamos apenas conferir se os "pares" (ex: `estado` / `estadoNotificacao`) contém o mesmo conteúdo.

In [23]:
len(df['estado'].unique()) == len(df['estadoNotificacao'].unique())

False

In [24]:
len(df['estado']) == len(df['estadoNotificacao'])

True

Há algo de diferente aqui.

Apesar de possuirem o mesmo número de registros, temos diferentes quantidade de valores únicos.

In [25]:
print(df['estado'].unique())

['Rio de Janeiro' 'Ceará' 'Rio Grande do Sul' 'Santa Catarina'
 'Minas Gerais' 'Pará' 'Piauí' 'Mato Grosso do Sul' 'Goiás' 'Rondônia'
 'Acre' 'Amazonas' 'Pernambuco' 'Paraná' 'Maranhão' 'Bahia' 'São Paulo'
 'Paraíba' 'Alagoas' 'Mato Grosso' 'Sergipe' 'Distrito Federal'
 'Tocantins' 'Rio Grande do Norte' nan 'Amapá' 'Espírito Santo' 'Roraima'
 'GOIAS']


In [26]:
print(df['estadoNotificacao'].unique())

['Rio de Janeiro' 'Ceará' 'Rio Grande do Sul' 'Santa Catarina'
 'Minas Gerais' 'Pará' 'Piauí' 'Mato Grosso do Sul' 'Goiás' 'Rondônia'
 'Acre' 'Amazonas' 'Pernambuco' 'Paraná' 'Maranhão' 'Bahia' 'São Paulo'
 'Paraíba' 'Alagoas' 'Mato Grosso' 'Sergipe' 'Distrito Federal'
 'Tocantins' 'Rio Grande do Norte' 'Amapá' 'Espírito Santo' 'Roraima'
 'GOIAS']


Em ambos o estado de Goiás está registrado duas vezes. Como "Goiás" e "GOIAS". Isso será fácil de corrigir.

Mas vemos algo além. `estado` possui alguns campos listados como `nan` *("Not a Number" nesse contexto é referente a um campo nulo)*. Vamos verificar quais são esses registros.

In [27]:
df[df['estado'].isnull()]

,Unnamed: 0,_id,dataNotificacao,cnes,ocupacaoSuspeitoCli,ocupacaoSuspeitoUti,ocupacaoConfirmadoCli,ocupacaoConfirmadoUti,ocupacaoCovidUti,ocupacaoCovidCli,...,origem,_p_usuario,estadoNotificacao,municipioNotificacao,estado,municipio,excluido,validado,_created_at,_updated_at
4182,4184,bc66V3say8,2020-04-24T12:32:29.807Z,2246902,1.0,0.0,1.0,0.0,NaN,NaN,...,aplicacao-web,_User$Ow1jWbSgxK,Rio Grande do Sul,Não-Me-Toque,NaN,NaN,False,False,2020-04-24T12:33:53.952Z,2020-04-24T12:33:53.952Z


Apena um registro tem campos faltantes em `estado` e `municipio`. Porém seus "pares" contam esses dados.

In [28]:
df.drop(4182, inplace=True)

Poderiamos dar esse caso em particular como resolvido, no entanto, será que todos os outros "pares" apresentam as mesmas informações?

In [29]:
len(df.loc[df['estado'] == df['estadoNotificacao']]) == df.shape[0]

False

Aperentemente não. Vejamos o motivo:

In [30]:
df.loc[df['estado'] != df['estadoNotificacao'], ['cnes', 'municipio', 'estado' , 'municipioNotificacao', 'estadoNotificacao']]

,cnes,municipio,estado,municipioNotificacao,estadoNotificacao
931,2438313,Minaçu,Goiás,Itaobim,Minas Gerais
2962,703208645495691,Buritis,Rondônia,São Romão,Minas Gerais
33764,223505,Cassilândia,Mato Grosso do Sul,Ouro Preto do Oeste,Rondônia
121811,2786095.0,Dianópolis,Tocantins,Brasília,Distrito Federal
193605,4069803,Juína,Mato Grosso,Propriá,Sergipe
438983,2013029,Manaus,Amazonas,Tunápolis,Santa Catarina
478919,3004104,Manaus,Amazonas,Propriá,Sergipe
516046,2017555,Carauari,Amazonas,Codó,Maranhão


Vemos algo sem coerência. Poderíamos discutir sobre o significado de cada coluna. Porém, dificilmente teríamos certeza do que se tratam *(ex: residente de um estado sendo notificado em outro durante uma viagem)*. 

Dessa forma, será melhor excluir os registros da análise. Também poderíamos argumentar que 8 registros são uma parcela muito pequena do arquivo.

In [31]:
print(f"8 registros representam {100 * (8 / df.shape[0]):.4f}% do arquivo")

8 registros representam 0.0014% do arquivo


In [32]:
df.drop(df.loc[df['estado'] != df['estadoNotificacao']].index, inplace=True)

len(df.loc[df['estado'] == df['estadoNotificacao']]) == df.shape[0]

True

Agora `estado` e `estadoNotificacao` são idênticas, podemos utilizar tanto uma quanto outra. Poderíamos executar o mesmo processo para `municipio` e `municipioNotificacao`, mas vamos fazer outra consideração antes.

### Consideração Sobre o Escopo da Análise



**Qual o escopo da análise?**

A idéia principal é de analisar o nível de ocupação hospitalar **em nível nacional**, portanto, talvez não seja interessante aumentarmos a *"resolução"* para municípios. Incorremos em **obscurecer nuances pelo exagero de detalhes**.

O que nos leva a considerar quais colunas são mais apropriadas e quais são menos compatíveis com o **objetivo da análise**.

Pelo menos em um primeiro momento, as informações mais relevantes seriam `estado`, `dataNotificacao`, `ocupcacaoSuspeitoCli`, `ocupacaoSuspeitoUti`, `ocupacaoConfirmadoCli`, `ocupacaoConfirmadoUti`, `saidaSuspeitaAltas`, `saidaSuspeitaObitos`, `saidaConfirmadaAltas`, `saidaConfirmadaObitos`. 

Campos como `municipio` e `cnes` entregam detalhes fora do escopo desta análise. Porém, são bons dados para análises subsequentes. *Por exemplo, após verificarmos os estados mais e menos afetados, poderíamos examiná-los com mais detalhes.*

In [33]:
print(f"No momento, o tamanho do nosso DataFrame é {(df.memory_usage().sum() / 10**6):.1f} Mb")

No momento, o tamanho do nosso DataFrame é 111.9 Mb


Temos algumas opções para separar os dados sobre municípios, incluindo bancos de dados relacionais para poupar armazenamento. 

No atual estado, o DataFrame tem aproximadamente 112 Mb. Não é um arquivo grande, mas de toda maneira, após a remoção das colunas marcadas o tamanho será consideravelmente reduzido. Quando fizermos isso, vamos considerar nossas opções.

Por último vamos apenas adicionar `estadoNotificacao` em nossa lista para remoção. *Como ainda não vasculhamos os municípios, vamos deixá-los como estão por enquanto*

In [34]:
to_drop.append('estadoNotificacao')
print(to_drop)

['_id', '_p_usuario', '_created_at', '_updated_at', 'origem', 'excluido', 'validado', 'estadoNotificacao']


Estas são algumas das colunas que vamos eliminar. Iremos fazer isso em breve, mas por enquanto temos utilidade para elas.

### Identificando Anomalias

Ainda precisamos averiguar os dados das **colunas numéricas**, e caso seja necessário, teremos que decidir como lidar com **dados anormais**.

Por isso havíamos separado as colunas para descarte em `to_drop` e `to_drop_numeric`. Colunas com variáveis categóricas *(`to_drop`)* trazem **contexto** para os registros, facilitando na decisão de **como lidar com valores faltantes**.

Já as numéricas irão apenas se confundir com as colunas que iremos de fato utilizar. Por isso agora serão removidas. 

In [35]:
df.drop(columns=to_drop_numeric, inplace=True)

Vamos novamente calcular as estatísticas descritivas.

In [36]:
df.describe()

,ocupacaoSuspeitoCli,ocupacaoSuspeitoUti,ocupacaoConfirmadoCli,ocupacaoConfirmadoUti,saidaSuspeitaObitos,saidaSuspeitaAltas,saidaConfirmadaObitos,saidaConfirmadaAltas
count,5.537720e+05,553772.000000,553773.000000,553773.000000,554057.000000,5.540690e+05,554161.000000,5.541600e+05
mean,1.902687e+02,7.141221,6.084807,4.480354,0.224684,5.143359e+01,0.373303,3.398676e+01
std,6.211689e+04,16.092486,33.192898,11.258679,3.572660,3.768619e+04,3.746363,1.730111e+04
min,-5.700000e+01,-9.000000,-10.000000,-7.000000,-10.000000,-1.200000e+01,-3.000000,-1.300000e+01
25%,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00
50%,4.000000e+00,0.000000,1.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00
75%,2.700000e+01,8.000000,6.000000,5.000000,0.000000,0.000000e+00,0.000000,0.000000e+00
max,2.908202e+07,924.000000,22222.000000,334.000000,425.000000,2.805202e+07,322.000000,9.112020e+06


Em primeira vista podemos notar alguns pontos mais chamativos, como:
* **Outliers**
* **Valores mínimos negativos**
* **Diferentes quantidades de registros**

Algo que também pode nos causar problemas são **dados contínuos**. Podemos checar se existe alguma ocorrência realizando a somatória de alguma coluna *(ou todas)* e verificando se obtemos algum número fracionado. 

In [37]:
df['ocupacaoSuspeitoCli'].sum()

105365479.64199996

Temos de fato valores contínuos em um ou mais registros.

Assim sendo, **iremos percorrer todos os pontos citados.**

#### Outliers

Como *29.082.020* parece um número exagerado de pacientes em um hospital, talvez seja um bom lugar para começar.

In [38]:
df['ocupacaoSuspeitoCli'].max()

29082020.0

Aqui temos um valor absolutamente fora da realidade. Na verdade parece um erro de digitação onde uma data foi inserida *(provavelmente 29/08/2020)*. De toda maneira, *não podemos saber qual seria o valor real*, então iremos considerar como outlier que deve ser seguramente descartado.

Vemos outro valor da mesma ordem em `saidaSuspeitaAltas`, então talvez seja prudente averiguar ocorrências semelhantes.

**Dados Extra:**

Temos nossa hipótese do motivo do erro, mas talvez precisemos de mais contexto em relação à capacidade real de hospitais brasileiros.

Convenientemente, o [Portal Brasileiro de Dados Abertos](https://dados.gov.br/home) possui recursos que vão nos ajudar exatamente nisso. Temos dados sobre [hospitais e leitos](https://dados.gov.br/dados/conjuntos-dados/hospitais-e-leitos) dividido por ano. E para nossa sorte esses dados possuem [documentação](https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/Leitos_SUS/Dicion%C3%A1rio_Leito_hospitalar.pdf).

Para referência, a partir de agora vamos chamar nosso primeiro DataFrame *(dados sobre ocupação de leitos)* de "`ocupação`", e o segundo *(dados sobre hospitais e leitos em si)* de "`leitos`".

In [39]:
df_leitos = pd.read_csv('leitos-2020.csv')
df_leitos.head()

,COMP,REGIAO,UF,MUNICIPIO,MOTIVO DESABILITACAO,CNES,NOME ESTABELECIMENTO,RAZAO SOCIAL,TP_GESTAO,CO_TIPO_UNIDADE,...,UTI ADULTO - EXIST,UTI ADULTO - SUS,UTI PEDIATRICO - EXIST,UTI PEDIATRICO - SUS,UTI NEONATAL - EXIST,UTI NEONATAL - SUS,UTI QUEIMADO - EXIST,UTI QUEIMADO - SUS,UTI CORONARIANA - EXIST,UTI CORONARIANA - SUS
0,202001,NORDESTE,PE,CABO DE SANTO AGOSTINHO,NaN,27,CASA DE SAUDE SANTA HELENA,CASA DE SAUDE E MATERNIDADE SANTA HELENA LTDA,M,5,...,0,0,0,0,0,0,0,0,0,0
1,202002,NORDESTE,PE,CABO DE SANTO AGOSTINHO,NaN,27,CASA DE SAUDE SANTA HELENA,CASA DE SAUDE E MATERNIDADE SANTA HELENA LTDA,M,5,...,0,0,0,0,0,0,0,0,0,0
2,202003,NORDESTE,PE,CABO DE SANTO AGOSTINHO,NaN,27,CASA DE SAUDE SANTA HELENA,CASA DE SAUDE E MATERNIDADE SANTA HELENA LTDA,M,5,...,0,0,0,0,0,0,0,0,0,0
3,202004,NORDESTE,PE,CABO DE SANTO AGOSTINHO,NaN,27,CASA DE SAUDE SANTA HELENA,CASA DE SAUDE E MATERNIDADE SANTA HELENA LTDA,M,5,...,0,0,0,0,0,0,0,0,0,0
4,202005,NORDESTE,PE,CABO DE SANTO AGOSTINHO,NaN,27,CASA DE SAUDE SANTA HELENA,CASA DE SAUDE E MATERNIDADE SANTA HELENA LTDA,M,5,...,0,0,0,0,0,0,0,0,0,0


In [40]:
df_leitos.columns

Index(['COMP', 'REGIAO', 'UF', 'MUNICIPIO', 'MOTIVO DESABILITACAO', 'CNES',
       'NOME ESTABELECIMENTO', 'RAZAO SOCIAL', 'TP_GESTAO', 'CO_TIPO_UNIDADE',
       'DS_TIPO_UNIDADE', 'NATUREZA_JURIDICA', 'DESC_NATUREZA_JURIDICA',
       'NO_LOGRADOURO', 'NU_ENDERECO', 'NO_COMPLEMENTO', 'NO_BAIRRO', 'CO_CEP',
       'NU_TELEFONE', 'NO_EMAIL', 'LEITOS EXISTENTE', 'LEITOS SUS',
       'UTI TOTAL - EXIST', 'UTI TOTAL - SUS', 'UTI ADULTO - EXIST',
       'UTI ADULTO - SUS', 'UTI PEDIATRICO - EXIST', 'UTI PEDIATRICO - SUS',
       'UTI NEONATAL - EXIST', 'UTI NEONATAL - SUS', 'UTI QUEIMADO - EXIST',
       'UTI QUEIMADO - SUS', 'UTI CORONARIANA - EXIST',
       'UTI CORONARIANA - SUS'],
      dtype='object')

Como temos uma documentação para nos guiar, será mais fácil encontrar o que queremos no momento.

De acordo com os dados que possuímos sobre *ocupação de leitos*, acredito que devemos resgatar informações como `UF`, `MUNICIPIO`, `CNES`, `LEITOS EXISTENTES`, `LEITOS SUS`, `UTI TOTAL - EXIST`, `UTI TOTAL - SUS` (para checar se 'EXIST' inclui 'SUS') e `COMP` *(como cada registro se refere a uma combinação ano-mês, precisamos ter cuidado para não contabilizar o mesmo estabelecimento mais de uma vez)*. Vemos que existem subcategorias em UTI, mas os *totais* devem nos fornecer uma boa estimativa. 

Lembrando, o escopo da análise é ocupação hospitalar em nível *nacional*. Mas para não descartar dados teoricamente bons, vamos manter a precisão de municípios por enquanto. Pode nos ser útil em outro momento.

Visto que `leitos` denomina estados por UF, é uma boa hora para fazer o mesmo com os dados de `ocupação`. Aproveitamos para 'corrigir' as ocorrências 'GOIAS' e 'Goiás'. O código feito para isso está em um arquivo auxiliar `utils.py`, na mesma pasta do projeto.

In [41]:
# Adiciona coluna 'uf' no DataFrame de acordo com a coluna 'estado'
df['uf'] = df['estado'].apply(lambda x: utils.get_uf(x, utils.uf_dict))

Com a documentação de `leitos`, vemos que o campo `COMP` se refere ao ano e mês que o registro se refere.

In [42]:
df_leitos['COMP'].value_counts()

COMP
202009    7080
202008    7076
202007    7065
202010    7063
202012    7051
202011    7050
202006    7004
202005    6973
202004    6891
202003    6775
202002    6696
202001    6688
Name: count, dtype: int64

A notação que encontramos é **ano-mês**, no formato **YYYYMM**. Também podemos ver que nem todo mês conta com a mesma quantidade de registros, nos indicando que nem todos os estabelecimentos possuem registro dos 12 meses.

É uma boa informação para observar a variação de capacidade com o passar do tempo. Podemos reformular a disposição dessa informação para facilitar a análise. No entanto, uma vez que adquirimos estes dados para uma **estimativa da capacidade hospitalar** do país, nossa prioridade será apurar dados nesse sentido. Podemos terminar o que começamos com `ocupação` e depois trabalhar com `leitos`.

Por enquanto podemos fazer uma inspeção mais simples e verificar qual hospital brasileiro possui a maior capacidade, e com base nisso buscar outliers em `ocupação`.

Utilizaremos este método para segregar outliers por dois motivos:
* Temos a conveniência da informação em `leitos`
* Pela impossibilidade de utilizar medidas de *tendência central*, uma vez que **os dados não apresentam uma distribuição normal**.

In [43]:
df_leitos.describe()

,COMP,MOTIVO DESABILITACAO,CNES,CO_TIPO_UNIDADE,NATUREZA_JURIDICA,CO_CEP,LEITOS EXISTENTE,LEITOS SUS,UTI TOTAL - EXIST,UTI TOTAL - SUS,UTI ADULTO - EXIST,UTI ADULTO - SUS,UTI PEDIATRICO - EXIST,UTI PEDIATRICO - SUS,UTI NEONATAL - EXIST,UTI NEONATAL - SUS,UTI QUEIMADO - EXIST,UTI QUEIMADO - SUS,UTI CORONARIANA - EXIST,UTI CORONARIANA - SUS
count,83412.000000,0.0,8.341200e+04,83412.000000,83412.000000,8.341200e+04,83412.000000,83412.000000,83412.000000,83412.000000,83412.000000,83412.000000,83412.000000,83412.000000,83412.000000,83412.000000,83412.000000,83412.000000,83412.000000,83412.000000
mean,202006.563001,NaN,3.245406e+06,6.440021,2137.678344,5.384539e+07,72.977473,49.520405,6.758116,3.333669,4.538040,2.183271,0.708315,0.382139,1.323658,0.702345,0.033556,0.022671,0.154546,0.043243
std,3.437454,NaN,2.132226e+06,3.413594,1121.203730,2.562547e+07,96.818627,82.707782,16.396366,11.249390,11.450796,7.526894,3.062761,2.100633,4.868504,3.385400,0.480115,0.337448,1.567275,0.665474
min,202001.000000,NaN,2.700000e+01,5.000000,1015.000000,1.150011e+06,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,202004.000000,NaN,2.260050e+06,5.000000,1244.000000,3.554700e+07,20.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,202007.000000,NaN,2.524848e+06,5.000000,2054.000000,5.870007e+07,41.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,202010.000000,NaN,3.061388e+06,5.000000,3069.000000,7.502507e+07,88.000000,55.000000,8.000000,0.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,202012.000000,NaN,9.997784e+06,21.000000,3999.000000,9.998000e+07,1843.000000,1730.000000,362.000000,325.000000,216.000000,186.000000,80.000000,51.000000,118.000000,104.000000,14.000000,7.000000,66.000000,20.000000


In [44]:
df_leitos[['LEITOS EXISTENTE', 'LEITOS SUS', 'UTI TOTAL - EXIST', 'UTI TOTAL - SUS']]

,LEITOS EXISTENTE,LEITOS SUS,UTI TOTAL - EXIST,UTI TOTAL - SUS
0,64,0,0,0
1,64,0,0,0
2,64,0,0,0
3,64,0,0,0
4,64,0,0,0
...,...,...,...,...
83407,28,24,0,0
83408,28,24,0,0
83409,28,24,0,0
83410,28,24,0,0


Aparentemente não há uma distorção comprometedora no conjunto de dados sobre `leitos`. Também segundo o que pode ser observado *'EXIST'* **inclui** leitos listados em *'SUS'*, então não precisamos calcular a ocupação máxima. `LEITOS EXISTENTE` e `UTI TOTAL - EXIST` possuem o que queremos.

In [45]:
print(f"Máximo leitos: {df_leitos['LEITOS EXISTENTE'].max()}\nMáximo leitos UTI: {df_leitos['UTI TOTAL - EXIST'].max()}")

Máximo leitos: 1843
Máximo leitos UTI: 362


Como **não estamos mapeando cada hospital com sua respectiva capacidade**, nosso limite para consideração de outliers será a capacidade máxima do estabelecimento com mais leitos. Assim, uma eventual operação acima da capacidade em hospitais menores deverá ser contabilizada. 

Segundo os dados de `leitos`, temos as seguintes *capacidades máximas*:
* Clínicos: 1843 leitos
* UTI: 362 leitos

Outro ponto para considerarmos este limite para outliers é o contexto. Por exemplo, diferente do *volume de vendas* em um e-commerce, a *ocupação de um hospital* apresenta valores **pouco flexíveis**, impedindo que picos muito acentuados sejam considerados aceitaveis.

In [46]:
# 20 valores máximos encontrados para 'ocupacaoConfirmadoCli' + 'ocupacaoSuspeitoCli'
temp = (df['ocupacaoConfirmadoCli'] + df['ocupacaoSuspeitoCli']).value_counts()
temp.sort_index().index[-20:]

Index([    1094.0,     1095.0,     1096.0,     1100.0,     1102.0,     1110.0,
           1117.0,     1212.0,     1278.0,     1506.0,     1629.0,     2324.0,
           2953.0,    22224.0,    30894.0,   130147.0, 20062021.0, 21072020.0,
       21082021.0, 29082020.0],
      dtype='float64')

In [47]:
# 20 valores máximos encontrados para 'ocupacaoConfirmadoUti' + 'ocupacaoSuspeitoUti'
temp = (df['ocupacaoConfirmadoUti'] + df['ocupacaoSuspeitoUti']).value_counts()
temp.sort_index().index[-20:]

Index([418.0, 419.0, 420.0, 421.0, 424.0, 425.0, 426.0, 429.0, 432.0, 433.0,
       434.0, 435.0, 437.0, 439.0, 440.0, 442.0, 452.0, 464.0, 685.0, 933.0],
      dtype='float64')

Mais um detalhe. Nas listas acima temos os resultados da soma de casos suspeitos e confirmados em registros individuais. Se descontarmos os outliers, temos resultados que podem ser entendidos como as lotações máximas para leitos. 

A julgar pelos resultados anteriores, o **total de leitos ocupados** em determinada `cnes` se dá pela **soma** `ocupacaoSuspeitoCli` + `ocupacaoConfirmadoCli`. *E o mesmo para leitos de UTI*.

Podemos também selecionar apenas as colunas que nos interessam no momento.

In [48]:
cols = ['cnes', 'ocupacaoSuspeitoCli', 'ocupacaoSuspeitoUti', 'ocupacaoConfirmadoCli', 'ocupacaoConfirmadoUti', \
        'saidaSuspeitaObitos', 'saidaSuspeitaAltas', 'saidaConfirmadaObitos', 'saidaConfirmadaAltas']

Primeiro registros com outliers em *leitos clínicos*: 

In [49]:
df.loc[(df['ocupacaoSuspeitoCli'] + df['ocupacaoConfirmadoCli']) > 1843, cols]

,cnes,ocupacaoSuspeitoCli,ocupacaoSuspeitoUti,ocupacaoConfirmadoCli,ocupacaoConfirmadoUti,saidaSuspeitaObitos,saidaSuspeitaAltas,saidaConfirmadaObitos,saidaConfirmadaAltas
78255,13846.0,2240.0,157.0,84.0,52.0,1.0,1.0,2.0,9.0
81706,2168200.0,20062020.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
133204,2570300,2.0,0.0,22222.0,0.0,0.0,0.0,0.0,0.0
164127,2139030,21072020.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
215744,434,2944.0,3.0,9.0,1.0,1.0,5.0,0.0,1.0
227161,2587734,21082020.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
254558,2686945,29082020.0,41.0,0.0,0.0,0.0,0.0,0.0,0.0
548467,2576341,130144.0,52.0,3.0,8.0,0.0,0.0,0.0,0.0
550698,2006510,30864.0,64.0,30.0,21.0,2.0,2.0,0.0,6.0


In [50]:
# Verificando a capacidade e localização de um registro retornado
df_leitos.loc[df_leitos['CNES'] == 13846, ['COMP', 'LEITOS EXISTENTE', 'UTI TOTAL - EXIST', 'UF', 'MUNICIPIO']]


,COMP,LEITOS EXISTENTE,UTI TOTAL - EXIST,UF,MUNICIPIO
1390,202001,1032,362,PR,CAMPO LARGO
1391,202002,1032,362,PR,CAMPO LARGO
1392,202003,1070,362,PR,CAMPO LARGO
1393,202004,1070,362,PR,CAMPO LARGO
1394,202005,1070,362,PR,CAMPO LARGO
1395,202006,1070,362,PR,CAMPO LARGO
1396,202007,1070,362,PR,CAMPO LARGO
1397,202008,1070,362,PR,CAMPO LARGO
1398,202009,1070,362,PR,CAMPO LARGO
1399,202010,1070,362,PR,CAMPO LARGO


Para leitos clínicos encontramos apenas 9 registros.

Agora para UTIs:

In [51]:
df.loc[(df['ocupacaoSuspeitoUti'] + df['ocupacaoConfirmadoUti']) > 380, cols]

,cnes,ocupacaoSuspeitoCli,ocupacaoSuspeitoUti,ocupacaoConfirmadoCli,ocupacaoConfirmadoUti,saidaSuspeitaObitos,saidaSuspeitaAltas,saidaConfirmadaObitos,saidaConfirmadaAltas
44105,5718368,175.0,234.0,105.0,176.0,2.0,15.0,3.0,9.0
44107,5718368,175.0,234.0,105.0,176.0,5.0,24.0,0.0,0.0
47910,5718368,109.0,192.0,109.0,192.0,2.0,8.0,1.0,2.0
49023,5718368,102.0,192.0,102.0,192.0,2.0,7.0,2.0,10.0
57500,5718368,175.0,234.0,123.0,177.0,4.0,9.0,3.0,10.0
...,...,...,...,...,...,...,...,...,...
517022,2337800,21.0,666.0,4.0,19.0,0.0,0.0,0.0,1.0
521438,7257406,77.0,193.0,27.0,189.0,1.0,4.0,3.0,8.0
521472,7257406,73.0,196.0,24.0,191.0,1.0,3.0,7.0,3.0
524296,7257406,72.0,202.0,22.0,198.0,0.0,1.0,5.0,4.0


Para UTIs encontramos 90 registros.

Da mesma maneira, podemos considerar valores incorretos os que somam mais **altas** ou **obitos** do que a capacidade total.

In [52]:
df.loc[(df['saidaSuspeitaObitos'] + df['saidaSuspeitaAltas']\
        + df['saidaConfirmadaObitos'] + df['saidaConfirmadaAltas']) > 1843 + 380, cols]

,cnes,ocupacaoSuspeitoCli,ocupacaoSuspeitoUti,ocupacaoConfirmadoCli,ocupacaoConfirmadoUti,saidaSuspeitaObitos,saidaSuspeitaAltas,saidaConfirmadaObitos,saidaConfirmadaAltas
44202,706500323934799,0.0,0.0,1.0,0.0,0.0,28052020.0,0.0,1.0
190603,2534789,9.0,0.0,9.0,0.0,0.0,0.0,12.0,17800.0
277740,2442728,12.0,0.0,3.0,0.0,3.0,3020.0,1.0,3.0
554213,2401665,30.0,0.0,8.0,0.0,0.0,0.0,0.0,9112020.0
554214,2401665,30.0,0.0,8.0,0.0,0.0,0.0,0.0,9102020.0


Apenas 5 nesta pesquisa. 

Estes são todos os registros que possuem pelo menos um campo contendo outliers. Porém, como resolver a situação?
Basicamente Podemos:

* **Remover** os registros
* **Imputar** *(substituir)* outliers para preservar o restante do registro

Seja qual for a maneira com que vamos lidar com isso, devemos utilizar o **mesmo método** para **todos os registros** corrompidos que encontramos. 

Vimos que em relação à *saídas* e *leitos clínicos* temos poucos registros. Porém, temos mais presença de outliers associados à ocupação de *UTI*s.

Uma vez que devemos lidar com todos os casos da mesma maneira para evitar viés na análise, *imputar* os dados pode ser uma opção melhor para o caso. O motivo principal não seria simplesmente a quantidade de registros, mas também evitar a perda de outras informações que eles carregam.

Quando calculamos as estatísticas descritivas de `ocupação`, mesmo com os outliers às alterando um pouco, podemos notar que em todos os campos os valores dentro do **IQR** *(Inter Quartile Range)* são bem baixos. Mas focando em ocupação de leitos clínicos, notamos que limiares superiores mais "volumosos" giram em torno de **15** e **20**. *`ocupacaoConfirmadoCli` e `ocupacaoSuspeitoCli` respectivamente.*

In [53]:
# valores máximos sobre leitos clínicos atrelados aos outliers de UTI
df.loc[(df['ocupacaoSuspeitoUti'] + df['ocupacaoConfirmadoUti']) > 380, ['ocupacaoConfirmadoCli', 'ocupacaoSuspeitoCli']].max()

ocupacaoConfirmadoCli    169.0
ocupacaoSuspeitoCli      175.0
dtype: float64

Sabemos que não estamos lidando com uma distribuição normal, porém maiores valores encontrados juntos com outliers em ocupação de UTIs estão na ordem de **centenas**.

Estão **dentro do limite** que estipulamos para ocupação de *leitos clínicos*, ao mesmo tempo que se mostram bastante superiores aos limites do IQR.

Sendo assim, **iremos imputar os outliers** para **preservar a informação** que os acompanha.

Para **preservar os dados em boas condições atrelados aos outliers**, podemos **substituir apenas os outliers por dados ausentes *(NaN)***, e em seguida fazer a *imputação*. 

Faremos isso *após analisar outras anomalias*, como **valores negativos** e os que **originalmente são ausentes**.

Vamos fazer desta maneira principalmente para *não misturarmos dados que de fato estão ausentes com os que adicionaremos* artificialmente. **Uma vez que esta etapa for cumprida aplicaremos a substituição de outliers por *Nan***.  

#### Valores Mínimos Negativos


Dados negativos para ocupação hospitalar não parecem estar corretos, uma vez que temos colunas dedicadas para registrar tanto o número de entradas como de altas. Então nos resta presumir que sejam resultado de algum **erro** de digitação onde:
* O valor correto não apresentaria o sinal negativo
* Ou o esperado seria uma correção de qualquer valor negativo para 0

De toda forma, para manter uniformidade no processo, vamos utilizar o mesmo método aplicado em outliers. Ainda assim podemos verificar rapidamente o tamanho do problema.

In [54]:
cols.remove('cnes')

for col in cols:
    print(f"{col}: {len(df.loc[df[col] < 0])} valores negativos")

negative_vals = df[cols].lt(0).any(axis=1)

print(f"\nTotal de registros com ao menos 1 campo negativo: {negative_vals.value_counts()[1]}")

ocupacaoSuspeitoCli: 56 valores negativos
ocupacaoSuspeitoUti: 24 valores negativos
ocupacaoConfirmadoCli: 23 valores negativos
ocupacaoConfirmadoUti: 59 valores negativos
saidaSuspeitaObitos: 16 valores negativos
saidaSuspeitaAltas: 97 valores negativos
saidaConfirmadaObitos: 8 valores negativos
saidaConfirmadaAltas: 73 valores negativos

Total de registros com ao menos 1 campo negativo: 320


Temos mais registros problemáticos aqui do que com outliers. Vamos também verificar alguns desses registros:

In [55]:
df[cols][negative_vals]

,ocupacaoSuspeitoCli,ocupacaoSuspeitoUti,ocupacaoConfirmadoCli,ocupacaoConfirmadoUti,saidaSuspeitaObitos,saidaSuspeitaAltas,saidaConfirmadaObitos,saidaConfirmadaAltas
152,1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0
1010,3.0,0.0,0.0,-3.0,0.0,0.0,0.0,0.0
2833,50.0,22.0,2.0,0.0,0.0,-3.0,0.0,0.0
4016,1.0,0.0,1.0,-2.0,0.0,1.0,0.0,0.0
4589,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-2.0
...,...,...,...,...,...,...,...,...
534198,26.0,4.0,3.0,0.0,0.0,-2.0,0.0,1.0
536408,14.0,0.0,1.0,0.0,0.0,-1.0,0.0,0.0
546703,7.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0
548565,39.0,0.0,6.0,0.0,0.0,0.0,0.0,-1.0


In [56]:
df[cols][negative_vals].describe()

,ocupacaoSuspeitoCli,ocupacaoSuspeitoUti,ocupacaoConfirmadoCli,ocupacaoConfirmadoUti,saidaSuspeitaObitos,saidaSuspeitaAltas,saidaConfirmadaObitos,saidaConfirmadaAltas
count,320.000000,320.000000,320.000000,320.000000,320.00000,320.00000,320.000000,320.000000
mean,17.612500,3.803125,2.484375,1.178125,-0.10000,-0.50000,0.134375,-0.300000
std,33.386215,8.100969,4.825456,4.084855,0.97749,1.43839,1.197204,1.794453
min,-57.000000,-9.000000,-10.000000,-7.000000,-10.00000,-12.00000,-3.000000,-13.000000
25%,0.000000,0.000000,0.000000,0.000000,0.00000,-1.00000,0.000000,0.000000
50%,6.000000,0.000000,1.000000,0.000000,0.00000,0.00000,0.000000,0.000000
75%,22.000000,5.000000,3.000000,0.000000,0.00000,0.00000,0.000000,0.000000
max,317.000000,50.000000,30.000000,34.000000,3.00000,4.00000,19.000000,13.000000


Nossa segunda hipótese pode ser descartada, uma vez que vemos valores negativos registrados em conjunto com valores = 0 em várias ocasiões.

A primeira hipótese não parece tão inadequada, no entanto, temos motivos para utilizar o método de imputação.

Como já discutimos, seria mais *padronizado* manter o *mesmo método* utilizado para outliers. Mas, complementar a isso, temos o fato de que as estatísticas descritivas do subgrupo selecionado apresentam **valores bem próximos aos do conjunto completo**. Isso pode nos indicar que *não haverá grandes distorções* quando aplicarmos a imputação de valores.

#### Dados contínuos

Variáveis *contínuas* *(ou números 'quebrados')* não fazem sentido o contexto que estamos trabalhando. Não há maneira de um leito estar ocupado por **4 pacientes e meio**. Um paciente ocupa ou não uma vaga no leito, sem meio termo.

Por este motivo, apenas variáveis *discretas* *(números inteiros)* são aceitáveis como dados sobre ocupação.

Seguiremos com o método de imputação. Para isso iremos substituir qualquer valor incompatível com a análise por *NaN*. Após isso feito, podemos aproveitar e ajustar o tipo das colunas afetadas de *float64* para *int64*.

Mais uma vez, faremos este procedimento uma vez que analisarmos os dados ausentes originais.

#### Valores Ausentes

Vejamos agora a **diferença em quantidade de registros** (causada por **valores ausentes** *já presentes*). 

Primeiro vamos verificar com que frequência e em quais situações isto ocorre. 

In [57]:
df['ocupacaoSuspeitoCli'].isna().value_counts()

ocupacaoSuspeitoCli
False    553772
True        401
Name: count, dtype: int64

In [58]:
for col in cols:
    print(f"{col}: {df[col].isna().value_counts()[1]} valores ausentes")

ocupacaoSuspeitoCli: 401 valores ausentes
ocupacaoSuspeitoUti: 401 valores ausentes
ocupacaoConfirmadoCli: 400 valores ausentes
ocupacaoConfirmadoUti: 400 valores ausentes
saidaSuspeitaObitos: 116 valores ausentes
saidaSuspeitaAltas: 104 valores ausentes
saidaConfirmadaObitos: 12 valores ausentes
saidaConfirmadaAltas: 13 valores ausentes


In [59]:
df.loc[df['ocupacaoSuspeitoCli'].isna() == True]

,_id,dataNotificacao,cnes,ocupacaoSuspeitoCli,ocupacaoSuspeitoUti,ocupacaoConfirmadoCli,ocupacaoConfirmadoUti,saidaSuspeitaObitos,saidaSuspeitaAltas,saidaConfirmadaObitos,...,_p_usuario,estadoNotificacao,municipioNotificacao,estado,municipio,excluido,validado,_created_at,_updated_at,uf
225086,d1blzunFYI,2020-08-19T03:00:00.000Z,9161449,NaN,NaN,30.0,22.0,0.0,1.0,3.0,...,_User$PNmsshy0Z0,Maranhão,São Luís,Maranhão,São Luís,False,False,2020-08-20T22:46:47.524Z,2020-08-20T22:46:47.524Z,MA
228389,IhrZ1JOhV8,2020-08-20T03:00:00.000Z,9161449,NaN,NaN,40.0,23.0,1.0,1.0,1.0,...,_User$PNmsshy0Z0,Maranhão,São Luís,Maranhão,São Luís,False,False,2020-08-21T19:28:45.418Z,2020-08-21T19:28:45.418Z,MA
554199,zOxCnmQLxK,2020-04-29T18:14:09.410Z,86673,NaN,NaN,NaN,NaN,NaN,0.0,1.0,...,_User$HzwsO90KQW,Ceará,Fortaleza,Ceará,Fortaleza,False,False,2021-08-10T03:01:39.990Z,2021-08-10T03:01:39.990Z,CE
554200,BvSDS2XLqp,2020-07-07T10:09:37.833Z,107239,NaN,NaN,NaN,NaN,NaN,0.0,1.0,...,_User$HzwsO90KQW,Ceará,Cascavel,Ceará,Cascavel,False,False,2021-08-10T03:01:40.313Z,2021-08-10T03:01:40.313Z,CE
554201,IF15Y9SJuY,2020-08-13T09:53:20.703Z,133094,NaN,NaN,NaN,NaN,NaN,0.0,0.0,...,_User$HzwsO90KQW,Ceará,Juazeiro do Norte,Ceará,Juazeiro do Norte,False,False,2021-08-10T03:01:40.417Z,2021-08-10T03:01:40.417Z,CE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
554682,A0ZNmQ4RpB,2020-12-04T03:00:00.000Z,2218798,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,_User$991WHtuARH,Minas Gerais,Juiz de Fora,Minas Gerais,Juiz de Fora,False,False,2023-02-15T19:28:39.477Z,2023-02-15T19:28:39.477Z,MG
554683,8h7PVYTbOs,2020-03-08T03:00:00.000Z,9887725,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,_User$i3dkQgse7y,Rio de Janeiro,Belford Roxo,Rio de Janeiro,Belford Roxo,False,False,2023-03-08T12:03:10.135Z,2023-03-08T12:03:10.135Z,RJ
554684,0wykjq5hlB,2020-03-30T03:00:00.000Z,6249604,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,_User$xadpF4Syg5,Santa Catarina,Ipumirim,Santa Catarina,Ipumirim,False,False,2023-04-06T14:28:58.666Z,2023-04-06T14:28:58.666Z,SC
554685,EkbSG9doPD,2020-04-06T03:00:00.000Z,2526662,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,_User$x4fJhKt6sz,Ceará,Quixeré,Ceará,Quixeré,False,False,2023-04-12T12:57:54.509Z,2023-04-12T12:57:54.509Z,CE


Aqui buscamos a **melhor maneira de lidar com valores ausentes**. Foi exatamente por isso que **ainda não substituímos outliers e medidas negativas por valores nulos**. Do contrário, qualquer especulação estaria *seriamente comprometida*.

Voltando às ausências, existem formas de abordar o problema. Em primeiro lugar, apenas podemos agir de acordo com certas *premissas* em relação a causa das ausências. Discutiremos isso em breve quando abordarmos as *categorias* de dados faltantes.

O que deve nos guiar nesse tipo de situação é a distribuição dos dados e a aleatoriedade das ausências. Dependendo dos dados que possuímos, **há o risco de introduzirmos algum viés na análise** caso o procedimento adotado seja inadequado. Por isso, a prioridade agora é tentar **entender o motivo da ausência dos dados**.

In [60]:
# Contagem de valores faltantes por estado
df.loc[df['ocupacaoSuspeitoCli'].isna() == True, 'estado'].value_counts()

estado
Ceará                  204
Paraná                  65
Rio de Janeiro          37
Pará                    33
Santa Catarina          23
Maranhão                 9
Pernambuco               5
Minas Gerais             5
Bahia                    4
São Paulo                3
Rio Grande do Norte      3
Amazonas                 3
Mato Grosso              2
Rio Grande do Sul        2
Tocantins                2
Distrito Federal         1
Name: count, dtype: int64

In [61]:
# Quantidade total de '_p_usuario' no arquivo
len(df['_p_usuario'].unique())

3173

In [62]:
# Quantos '_p_usuario' fizeram registros com valores ausentes na coluna 'ocupacaoSuspeitoCli'
len(df.loc[df['ocupacaoSuspeitoCli'].isna() == True]['_p_usuario'].unique())

56

In [63]:
shape = df.shape[0]
print(f"Das {shape} linhas, {shape - df.dropna().shape[0]} delas tem pelo menos 1 valor ausente")

Das 554173 linhas, 569 delas tem pelo menos 1 valor ausente


Explorando essa parcela dos dados, é possível fazer algumas observações sobre os dados faltantes:
* **A grande maioria deles é referente ao estado do Ceará**
* 56 dos 3.173 `_p_usuario`*(ou 1,76%)* são autores dos registros incompletos
    * Ao passo que 569 dos 554.173 registros estão incompletos *(ou 0,1%)* 

Com base nisso, podemos dizer que os dados ausentes **não aparecem completamente ao acaso**, existe alguma relação com outras variáveis. Podemos dizer que há maior probabilidade de encontrarmos um registro incompleto *dependendo do estado e do autor*.

Mas como podemos descrever a situação?

Dados ausentes são classificados em 3 tipos:
* *MCAR* - Missing Completely At Random *(Ausente completamente por acaso)*
* *MAR* - Missing At Random *(Ausente por acaso)*
* *MNAR* - Missing Not At Random *(Ausente não por acaso)*

Estas denominações podem ser um pouco contraintuitivas, mas em resumo:
* *MCAR* - valores ausentes não possuem relação alguma com nada no conjunto de dados. Um processo puramente aleatório
* *MAR* - valores ausentes possuem algum tipo de relação com alguma outra variável 
* *MNAR* - A probabilidade de um dado ser ausente depende do próprio dado ausente

Verificamos que os dados faltantes **apresentam um padrão perceptível**, então podemos **eliminar a possibilidade de estarmos trabalhando com *MCAR***.


In [64]:
df.loc[df['estado'] == 'Ceará'].describe()

,ocupacaoSuspeitoCli,ocupacaoSuspeitoUti,ocupacaoConfirmadoCli,ocupacaoConfirmadoUti,saidaSuspeitaObitos,saidaSuspeitaAltas,saidaConfirmadaObitos,saidaConfirmadaAltas
count,15474.000000,15474.000000,15474.000000,15474.000000,15666.000000,15678.000000,15672.000000,15672.000000
mean,46.056546,10.338503,8.965038,5.845295,0.110303,0.452226,0.275523,0.713374
std,72.176976,23.662064,21.580584,15.653932,0.431551,1.318697,1.024562,2.457325
min,-3.000000,0.000000,-5.000000,-2.000000,-2.000000,-2.000000,-2.000000,-4.000000
25%,3.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,13.000000,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,66.000000,9.000000,8.000000,6.000000,0.000000,0.000000,0.000000,1.000000
max,394.000000,323.000000,333.000000,150.000000,8.000000,30.000000,23.000000,77.000000


In [65]:
df.loc[df['estado'] == 'Ceará', ['ocupacaoSuspeitoCli']].value_counts()

ocupacaoSuspeitoCli
0.0                    2128
1.0                     806
2.0                     795
10.0                    663
3.0                     655
                       ... 
290.0                     1
292.0                     1
297.0                     1
302.0                     1
394.0                     1
Name: count, Length: 324, dtype: int64

Ficamos entre *MAR* e *MNAR*.

De maneira geral, a causa mais comum para dados *MNAR* são *tópicos sensíveis*. 

Por exemplo, em uma pesquisa com pessoas sobre um tema qualquer. *Quanto mais sensível* ou invasiva for a questão, *maior é a probabilidade de não ser respondida*. Este é um clássico exemplo do que mencionamos antes: "a probabilidade de um dado ser ausente depende do próprio dado ausente".

Seguindo nosso contexto, não temos motivos para acreditar que a ausência seja motivada por algo mais complexo do que, por exemplo, um *erro de digitação*, por exemplo. Observamos algumas **situações** com outliers e valores negativos que **reforçam a idéia** da fonte das anomalias ser meramente algum tipo de **equívoco ou erro na entrada do registro**.

Além disso, nos baseamos na premissa de que os dados faltantes tem alguma *relação com as variáveis observadas*. Sendo assim, iremos considerar que os dados ausententes se enquadram na categoria **MAR**.

### Substituindo Dados Anômalos

A partir do momento que identificamos a categoria em que nossos dados faltantes se encaixam, podemos lidar com as outras anomalias. Como decidido, iremos **substituir** todas elas por ***NaN***.

Apenas para recordar, vamos listar todas as remoções que faremos:
* Outlier
* Valores negativos
* Valores contínuos

Após isso feito, seguiremos para a imputação de dados.

#### Outliers

Infelizmente, como utilizamos a **soma** de ocupação por casos *suspeitos* e *confirmados* para identificar outliers, acabamos por *atrelar* os dois valores. 

Isso significa que quando identificamos um registro com outlier, temos que remover todos os dados utilizados para chegar a esta conclusão. Por exemplo, se identificarmos um registro com outliers em leitos **clínicos**, precisamos substituir tanto `ocupacaoSuspeitoCli` quanto `ocupacaoConfirmadoCli`.

##### Leitos Clínicos

In [67]:
drop_indexes = df.loc[(df['ocupacaoSuspeitoCli'] + df['ocupacaoConfirmadoCli']) > 1843].index
drop_indexes

Index([78255, 81706, 133204, 164127, 215744, 227161, 254558, 548467, 550698], dtype='int64')

In [68]:
# Antes da modificação
df.loc[78255]

_id                                    yOXQI4OS0B
dataNotificacao          2020-06-19T03:00:00.000Z
cnes                                      13846.0
ocupacaoSuspeitoCli                        2240.0
ocupacaoSuspeitoUti                         157.0
ocupacaoConfirmadoCli                        84.0
ocupacaoConfirmadoUti                        52.0
saidaSuspeitaObitos                           1.0
saidaSuspeitaAltas                            1.0
saidaConfirmadaObitos                         2.0
saidaConfirmadaAltas                          9.0
origem                                parse-cloud
_p_usuario                       _User$PtPnlbdEeC
estadoNotificacao                          Paraná
municipioNotificacao                  Campo Largo
estado                                     Paraná
municipio                             Campo Largo
excluido                                    False
validado                                    False
_created_at              2020-06-19T21:54:36.096Z


In [69]:
target_cols = ['ocupacaoSuspeitoCli', 'ocupacaoConfirmadoCli']
for item in drop_indexes:
    for col in target_cols:
        df.loc[item, col] = np.NaN

In [70]:
# Após a modificação - 'ocupacaoSuspeitoCli' e 'ocupacaoSuspeitoCli' devem ser 'NaN'
df.loc[78255]

_id                                    yOXQI4OS0B
dataNotificacao          2020-06-19T03:00:00.000Z
cnes                                      13846.0
ocupacaoSuspeitoCli                           NaN
ocupacaoSuspeitoUti                         157.0
ocupacaoConfirmadoCli                         NaN
ocupacaoConfirmadoUti                        52.0
saidaSuspeitaObitos                           1.0
saidaSuspeitaAltas                            1.0
saidaConfirmadaObitos                         2.0
saidaConfirmadaAltas                          9.0
origem                                parse-cloud
_p_usuario                       _User$PtPnlbdEeC
estadoNotificacao                          Paraná
municipioNotificacao                  Campo Largo
estado                                     Paraná
municipio                             Campo Largo
excluido                                    False
validado                                    False
_created_at              2020-06-19T21:54:36.096Z


Apenas verificando se tudo deu certo:

In [71]:
df.loc[(df['ocupacaoSuspeitoCli'] + df['ocupacaoConfirmadoCli']) > 1843]

,_id,dataNotificacao,cnes,ocupacaoSuspeitoCli,ocupacaoSuspeitoUti,ocupacaoConfirmadoCli,ocupacaoConfirmadoUti,saidaSuspeitaObitos,saidaSuspeitaAltas,saidaConfirmadaObitos,...,_p_usuario,estadoNotificacao,municipioNotificacao,estado,municipio,excluido,validado,_created_at,_updated_at,uf


In [72]:
df.loc[drop_indexes, cols]

,ocupacaoSuspeitoCli,ocupacaoSuspeitoUti,ocupacaoConfirmadoCli,ocupacaoConfirmadoUti,saidaSuspeitaObitos,saidaSuspeitaAltas,saidaConfirmadaObitos,saidaConfirmadaAltas
78255,NaN,157.0,NaN,52.0,1.0,1.0,2.0,9.0
81706,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0
133204,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0
164127,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0
215744,NaN,3.0,NaN,1.0,1.0,5.0,0.0,1.0
227161,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0
254558,NaN,41.0,NaN,0.0,0.0,0.0,0.0,0.0
548467,NaN,52.0,NaN,8.0,0.0,0.0,0.0,0.0
550698,NaN,64.0,NaN,21.0,2.0,2.0,0.0,6.0


Perfeito. Vamos fazer o mesmo procedimento com leitos de UTI e saídas.

##### Leitos UTI

In [73]:
drop_indexes = df.loc[(df['ocupacaoSuspeitoUti'] + df['ocupacaoConfirmadoUti']) > 380].index
target_cols = ['ocupacaoSuspeitoUti', 'ocupacaoConfirmadoUti']
for item in drop_indexes:
    for col in target_cols:
        df.loc[item, col] = np.NaN

In [74]:
df.loc[(df['ocupacaoSuspeitoUti'] + df['ocupacaoConfirmadoUti']) > 380, cols]

,ocupacaoSuspeitoCli,ocupacaoSuspeitoUti,ocupacaoConfirmadoCli,ocupacaoConfirmadoUti,saidaSuspeitaObitos,saidaSuspeitaAltas,saidaConfirmadaObitos,saidaConfirmadaAltas


In [75]:
df.loc[drop_indexes, cols]

,ocupacaoSuspeitoCli,ocupacaoSuspeitoUti,ocupacaoConfirmadoCli,ocupacaoConfirmadoUti,saidaSuspeitaObitos,saidaSuspeitaAltas,saidaConfirmadaObitos,saidaConfirmadaAltas
44105,175.0,NaN,105.0,NaN,2.0,15.0,3.0,9.0
44107,175.0,NaN,105.0,NaN,5.0,24.0,0.0,0.0
47910,109.0,NaN,109.0,NaN,2.0,8.0,1.0,2.0
49023,102.0,NaN,102.0,NaN,2.0,7.0,2.0,10.0
57500,175.0,NaN,123.0,NaN,4.0,9.0,3.0,10.0
...,...,...,...,...,...,...,...,...
517022,21.0,NaN,4.0,NaN,0.0,0.0,0.0,1.0
521438,77.0,NaN,27.0,NaN,1.0,4.0,3.0,8.0
521472,73.0,NaN,24.0,NaN,1.0,3.0,7.0,3.0
524296,72.0,NaN,22.0,NaN,0.0,1.0,5.0,4.0


##### Altas

In [76]:
drop_indexes = df.loc[(df['saidaSuspeitaObitos'] + df['saidaSuspeitaAltas'] + df['saidaConfirmadaObitos']\
                        + df['saidaConfirmadaAltas']) > 1843 + 380].index
target_cols = ['saidaSuspeitaObitos', 'saidaSuspeitaAltas', 'saidaConfirmadaObitos', 'saidaConfirmadaAltas']
for item in drop_indexes:
    for col in target_cols:
        df.loc[item, col] = np.NaN

In [77]:
df.loc[(df['saidaSuspeitaObitos'] + df['saidaSuspeitaAltas'] + df['saidaConfirmadaObitos'] + df['saidaConfirmadaAltas']) > 1843 + 380]

,_id,dataNotificacao,cnes,ocupacaoSuspeitoCli,ocupacaoSuspeitoUti,ocupacaoConfirmadoCli,ocupacaoConfirmadoUti,saidaSuspeitaObitos,saidaSuspeitaAltas,saidaConfirmadaObitos,...,_p_usuario,estadoNotificacao,municipioNotificacao,estado,municipio,excluido,validado,_created_at,_updated_at,uf


In [78]:
df.loc[drop_indexes, cols]

,ocupacaoSuspeitoCli,ocupacaoSuspeitoUti,ocupacaoConfirmadoCli,ocupacaoConfirmadoUti,saidaSuspeitaObitos,saidaSuspeitaAltas,saidaConfirmadaObitos,saidaConfirmadaAltas
44202,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN
190603,9.0,0.0,9.0,0.0,NaN,NaN,NaN,NaN
277740,12.0,0.0,3.0,0.0,NaN,NaN,NaN,NaN
554213,30.0,0.0,8.0,0.0,NaN,NaN,NaN,NaN
554214,30.0,0.0,8.0,0.0,NaN,NaN,NaN,NaN


Terminamos com outliers.

#### Valores Negativos

Seguiremos o mesmo princípio para valores negativos.

In [79]:
#for col in cols:
#    print(f"{col}: {len(df.loc[df[col] < 0])} valores negativos")

#negative_vals = df[cols].lt(0).any(axis=1)

#### Valores Contínuos

In [80]:
#for col in cols:
#    df[col] = df[col].apply(lambda x: utils.check_int(x))

### Imputando Dados

### Reduzindo o Arquivo

E finalmente vamos excluir todas as outras colunas que foram selecionadas.

In [81]:
#df.drop(columns=to_drop, inplace=True)
#print(f"Agora nosso DataFrame tem {(df.memory_usage().sum() / 10**6):.1f} Mb")

Quase metade do tamanho original. E ainda não eliminamos as colunas relacionadas a municípios.

Podemos fazer isso sem prejuízo, mas antes, como podemos utilizar esses dados em **outra análises**, vamos salvar o DataFrame como um **novo arquivo**.

Considerando que temos um *arquivo pequeno* e não estamos reduzindo espaço de armazenamento a todo custo, transformar nossos dados atuais em tabelas em um banco de dados relacional pode não ser a solução mais prática. No lugar disso, podemos apenas *salvar o estado atual do DataFrame no formato `.csv`*.

In [82]:
# Descomentar para salvar o arquivo
#df.to_csv('ocupacao-leito-2020-municipios.csv', index=False, encoding='utf-8')

Com isso feito, podemos retirar as colunas restantes.

In [83]:
#df.drop(columns=['cnes', 'municipio', 'municipioNotificacao'], inplace=True)

Apenas por curiosidade, podemos checar o quanto reduzimos o arquivo.

In [84]:
#print(f"Agora nosso DataFrame tem {(df.memory_usage().sum() / 10**6):.1f} Mb")